In [1]:
import os
import copy
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from textwrap import dedent
import logging
import datetime
from pathlib import Path

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
# print(f'Package version: {llmt.__version__}')

### Load the data ###

In [2]:
date_str = datetime.date.today().strftime('%y%m%d')
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')
xls_file_1 = 'inpatient-companies-classification-03.18.2024.xlsx'
xls_file_2 = 'inpatient-companies-classification-Additional negs-04.01.2025.xlsx'

# Create a new data frame with the cleaned-up data frame
output_file_name = f'hcp-alldata-250413.parquet'
output_file = os.path.join(data_dir, output_file_name)

df1 = pd.read_excel(os.path.join(data_dir, xls_file_1))
df2 = pd.read_excel(os.path.join(data_dir, xls_file_2))

print(df1.shape)
print(df2.shape)

(2025, 10)
(35, 10)


In [3]:
# Clean up and rename the columns
def clean_df(data, labeled_rows='train', un_labeled_rows='test'):
    df = copy.deepcopy(data).\
                    rename(columns={'Companies': 'name',
                                    'CompanyID': 'id',
                                    'Description': 'description',
                                    'inpatient_healthcare ': 'inpatient',
                                    'outpatient_healthcare': 'outpatient'})
    cols = ['id', 'name', 'description', 'mental_health', 'inpatient', 'outpatient']
    df = df[cols]
    
    # Select one row per id
    n_id_before = len(df['id'].unique())
    df = df.drop_duplicates(subset='id', keep='first', ignore_index=True)
    n_id_after = len(df['id'].unique())
    assert n_id_before == n_id_after
    df = df.assign(dset=None)
    df.loc[~df['mental_health'].isnull(), 'dset'] = labeled_rows
    df.loc[df['mental_health'].isnull(), 'dset'] = un_labeled_rows
    return df

### Check the labels for the duplicate IDs ###

In [4]:
df1_cleaned = clean_df(data=df1)
df2_cleaned = clean_df(data=df2)
df1_labeled = df1_cleaned.loc[df1_cleaned['dset'] == 'train']
df2_labeled = df2_cleaned.loc[df2_cleaned['dset'] == 'train']

# The company IDs are unique
print(df1_labeled.shape)
print(len(df1_labeled['id'].unique()))
print(df2_labeled.shape)
print(len(df2_labeled['id'].unique()))
duplicate_id_list = []
df2_id_list = df2_labeled['id'].unique()
for id2 in df2_id_list:
    df1_id = df1_labeled.loc[df1_labeled['id'] == id2]
    if len(df1_id) > 0:
        duplicate_id_list.append(id2)
        df2_id = df2_labeled.loc[df2_labeled['id'] == id2]
#        display(df1_id)
#        display(df2_id)

# Are the duplicate rows are the same
d1 = df1_labeled.loc[df1_labeled['id'].isin(duplicate_id_list)].\
                sort_values(by='id', ascending=True).\
                reset_index(drop=True).\
                astype({'inpatient': int, 'outpatient': int, 'mental_health': int})
d2 = df2_labeled.loc[df2_labeled['id'].isin(duplicate_id_list)].\
                sort_values(by='id', ascending=True).\
                reset_index(drop=True)

print(f'The labels for the duplicate IDs are the same: {d1.equals(d2)}')

(169, 7)
169
(35, 7)
35
The labels for the duplicate IDs are the same: True


In [5]:
# Combine the data sets
df = pd.concat([df1_cleaned, df2_cleaned], axis=0, ignore_index=True)

# Get the training samples with labels
df_train = df.loc[df['dset'] == 'train'].\
                drop_duplicates().\
                reset_index(drop=True)
df_train_id_list = list(df_train['id'].unique())

# Get the test samples without labels
df_test = df.loc[df['dset'] == 'test'].\
                drop_duplicates().\
                reset_index(drop=True)
df_test_id_list = list(df_test['id'].unique())

# Remove the ids from this list where we already have labels
df_test_id_list_diff = set(df_test_id_list).difference(df_train_id_list)

print(f'Number of IDs in the test set before removing duplicates: {len(df_test_id_list)}')
print(f'Number of IDs in the test set after removing duplicates:  {len(df_test_id_list_diff)}')
print(f'Removing {len(df_test_id_list) - len(df_test_id_list_diff)} duplicate samples from test set.')
df_test = df_test[df_test['id'].isin(df_test_id_list_diff)]

# Combine the train and test sets
df_combined = pd.concat([df_train, df_test], axis=0, ignore_index=True)

# Make sure that the ids are unique
print(len(df_combined['id'].unique()))
print(df_combined.shape)

Number of IDs in the test set before removing duplicates: 1856
Number of IDs in the test set after removing duplicates:  1838
Removing 18 duplicate samples from test set.
2025
(2025, 7)


In [6]:
display(df_combined.head())

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2.0,0.0,0.0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0.0,0.0,0.0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0.0,0.0,0.0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1.0,0.0,1.0,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2.0,0.0,0.0,train


In [7]:
df_test = df_combined.loc[df_combined['mental_health'].isnull()]
df_train = df_combined.loc[~df_combined['mental_health'].isnull()]
print(df_test.shape)
print(df_train.shape)

(1838, 7)
(187, 7)


In [8]:
136 + 35 + 16

187

In [9]:
1838 + 187

2025

In [10]:
# Count the labels
label_list = ['mental_health', 'inpatient', 'outpatient']
for label in label_list:
    print(label.upper())
    df_label = df_combined[label].\
                    value_counts().\
                    to_frame().\
                    reset_index(drop=False).\
                    sort_values(by=label, ascending=False).\
                    reset_index(drop=True)
    display(df_label)

MENTAL_HEALTH


,mental_health,count
0,2.0,16
1,1.0,136
2,0.0,35


INPATIENT


,inpatient,count
0,2.0,5
1,1.0,69
2,0.0,113


OUTPATIENT


,outpatient,count
0,2.0,51
1,1.0,94
2,0.0,42


In [11]:
# Save the final data frame
df_combined.to_parquet(output_file)
display(df_combined.head())
print()
display(df_combined.sample(5))
print(output_file)

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2.0,0.0,0.0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0.0,0.0,0.0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0.0,0.0,0.0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1.0,0.0,1.0,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2.0,0.0,0.0,train


,id,name,description,mental_health,inpatient,outpatient,dset
1768,234369-91,Treatment Partners of America,Operator of a addiction and dual diagnosis tre...,NaN,NaN,NaN,test
1521,121179-70,Saint Clare's Health,Operator of community hospitals and healthcare...,NaN,NaN,NaN,test
114,60837-40,Prime Healthcare Services,Provider of medical and healthcare services in...,1.0,1.0,1.0,train
452,285754-15,Carolina Pain And Weight,"Provider of pain management, weight loss, and ...",NaN,NaN,NaN,test
1833,310185-01,Verdugo Home Health,Provider of home health care services to indiv...,NaN,NaN,NaN,test


/app/home_data/hcp/hcp-alldata-250413.parquet
